In [1]:
import pandas as pd
import numpy as np

import os, glob

In [2]:
path = os.path.join(os.path.dirname(os.getcwd()), '../Data/Extracted_data/')
os.chdir(path)

In [3]:
#Fracture toughness
FT = pd.read_csv('fracture_toughness.csv')
FT.set_index('ID', inplace=True)
FT.drop(['Type', 'Percent_cold_work','KIC_stress_intensity_max_ksi_x_in1_2', 'KIC_stress_intensity_range_ksi_x_in1_2'], axis=1, inplace=True)

#TEP (both)
TEP = pd.read_csv('TEP_magnetic_susceptibility_summary.csv')
TEP.set_index('ID', inplace=True)
TEP.drop(['material', 'cold_work_percent'], axis=1, inplace=True)

#Nonlinear
UT_NL = pd.read_csv('UT_nonlinear.csv')
UT_NL.set_index('ID', inplace=True)
UT_NL.drop(['material', 'cold_work_percent'], axis=1, inplace=True)
UT_NL = UT_NL.groupby('ID').mean().add_suffix('_avg')

In [7]:
#Internal Friction 25MHz
IF_25_dfs=[]
for file in glob.glob('*friction*25MHz.csv'):
    df = pd.read_csv(file)
    IF_25_dfs.append(df)

UT_IF_25 = pd.concat(IF_25_dfs, axis=0, ignore_index=False)
UT_IF_25['Sample ID'] = UT_IF_25['Sample ID'].str.replace('%|CW|\,', '').str.strip().str.replace(' - ', '-')\
.str.replace('  ', ' ').str.replace(' ', '-').str.strip('-1')+ '-1'
UT_IF_25.drop(['% CW'], axis=1, inplace=True)
UT_IF_25.rename(columns={'Sample ID':'ID'}, inplace=True)
UT_IF_25.set_index('ID', inplace=True)
UT_IF_25 = UT_IF_25.add_suffix('_2.25MHz')

In [8]:
#Internal Friction 5MHz
IF_5_dfs=[]
for file in glob.glob('*friction*_5MHz.csv'):
    df = pd.read_csv(file)
    IF_5_dfs.append(df)

UT_IF_5 = pd.concat(IF_5_dfs, axis=0, ignore_index=False)
UT_IF_5['Sample ID'] = UT_IF_5['Sample ID'].str.replace('%|CW|\,', '').str.strip().str.replace(' - ', '-')\
.str.replace('  ', ' ').str.replace(' ', '-').str.strip('-1')+ '-1'
UT_IF_5.drop(['% CW'], axis=1, inplace=True)
UT_IF_5.rename(columns={'Sample ID':'ID'}, inplace=True)
UT_IF_5.set_index('ID', inplace=True)
UT_IF_5 = UT_IF_5.add_suffix('_3.5MHz')

In [9]:
#Backscattering
UT_BS = pd.read_csv('UT_backscattering.csv')

UT_BS['Sample ID'] = UT_BS['Sample ID'].str.replace('%|CW|\,', '').str.strip().str.replace(' - ', '-')\
.str.replace('  ', ' ').str.replace(' ', '-').str.strip('-1')+ '-1'
UT_BS.rename(columns={'Sample ID':'ID'}, inplace=True)
UT_BS.set_index('ID', inplace=True)

In [10]:
merged = FT.join([TEP, UT_NL, UT_IF_25, UT_IF_5, UT_BS], how='left')

In [11]:
merged.to_csv('MERGE_FT_TEP_UT.csv')